# Merging Data from 2005 to 2020 for : *Usagers* & *Vehicules*
## Import libraries & packages

In [85]:
import pandas as pd
import numpy as np

## Load CSV files : 2005..2020

In [86]:
separator= ','
tiret='-'

for year in range(2005, 2021):

#    if year> 2016:
#        tiret='-'

    if year> 2018:
        separator=';'

 #   if year!= 2009:
 #       locals()["carac"+"_"+str(year)[-2:]] = pd.read_csv('data/'+str(year)+'/caracteristiques'+tiret + str(year) +'.csv', sep=separator)
    
#    if year== 2009:
#        locals()["carac"+"_"+str(year)[-2:]] = pd.read_csv('data/'+str(year)+'/caracteristiques'+tiret+ str(year) +'.csv', sep='	')
    
#    locals()["lieux"+"_"+str(year)[-2:]] = pd.read_csv('data/'+str(year)+'/lieux'+tiret+ str(year) +'.csv', sep= separator)
    locals()["veh"+"_"+str(year)[-2:]] = pd.read_csv('data/'+str(year)+'/vehicules'+tiret+ str(year) +'.csv', sep= separator)
    locals()["user"+"_"+str(year)[-2:]] = pd.read_csv('data/'+str(year)+'/usagers'+tiret+ str(year) +'.csv', sep= separator)

### Adjusting and cleaning *Usagers* tables

* There is a diffrence between *Usagers* tables before and after 2019:

In [87]:
print(list(user_19.columns))
print(list(user_18.columns))

['Num_Acc', 'id_vehicule', 'num_veh', 'place', 'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2', 'secu3', 'locp', 'actp', 'etatp']
['Num_Acc', 'place', 'catu', 'grav', 'sexe', 'trajet', 'secu', 'locp', 'actp', 'etatp', 'an_nais', 'num_veh']


* The diffrences are the *'id_vehicule'* and security columns : *'secu'*, *'secu1'*, *'secu2'* & *'secu3'*. The columns in every year must be the same, this is how we will adapt btween the years before and the years after 2019 :
    - We drop secu2 and secu3 from user_19 and user_20, since they don't exist in the other years, besides most of the values in secu3 and a third of secu2 values are not specified.
    - We rename *secu1* to *secu*, to match the years before 2019.
    - We adjust values in 2019-2020 to the values of the years before.
    - *Usagers* table in years 2005..2018 has only one column for security equipment with the name of *secu*, we will fit it with the 2019-2020 labeling.


In [88]:
#droping id_vehicule, secu2 & secu3 in 219-2020:
user_19.drop(['id_vehicule','secu2', 'secu3'], axis=1, inplace= True)
user_20.drop(['id_vehicule', 'secu2', 'secu3'], axis=1, inplace= True)

#rename secu1 columns:
user_19.rename( columns={'secu1':'secu'}, inplace=True)
user_20.rename( columns={'secu1':'secu'}, inplace=True)

#Adjust values in 2019-2020 to match the labeling in =<2018
user_19.secu.replace([[5,6,7], 8], [9, -1], inplace=True )
user_20.secu.replace([[5,6,7], 8], [9, -1], inplace=True )

In [89]:
#define the secu adjusting function:
def adjust_secu_with_2019(x):
    if len(str(x))!=2 or str(x)[1]=='3' or x==-1 :
        y=-1
    elif str(x)[1]=='2':
        y=0
    else:
        y= int(str(x)[0])
    return y

In [90]:
#applying the secu adjusting function on Usagers tables from 2005 to 2018:

for year in range(2005, 2019):
    
    #filling NaN with (-1)
    locals()["user"+"_"+str(year)[-2:]].secu =  locals()["user"+"_"+str(year)[-2:]].secu.fillna(-1)
    
    #change secu type from float to int
    locals()["user"+"_"+str(year)[-2:]].secu = locals()["user"+"_"+str(year)[-2:]].secu.astype({'secu' : 'int64'})
    
    #apply adjusting function
    locals()["user"+"_"+str(year)[-2:]].secu = locals()["user"+"_"+str(year)[-2:]].secu.apply(adjust_secu_with_2019)

* Most of the values in *actp* column are set to 0 (=not specified), so we drop it. The same applies for *actp* and *locp* columns:

In [91]:
for year in range(2005, 2021):
    locals()["user"+"_"+str(year)[-2:]].drop(['actp','etatp','locp'], axis=1, inplace= True )

### Adjusting  and cleaning *Vehicules* tables

* The diffrences in the *Vehicules* table before and after 2019:

In [92]:
print(list(veh_19.columns))
print(list(veh_05.columns))

['Num_Acc', 'id_vehicule', 'num_veh', 'senc', 'catv', 'obs', 'obsm', 'choc', 'manv', 'motor', 'occutc']
['Num_Acc', 'senc', 'catv', 'occutc', 'obs', 'obsm', 'choc', 'manv', 'num_veh']


* We add ***id_vehicule*** column to the *Vehicule* table for years between 2005 & 2018, this will make it easier to understand the relation woth other tables once we started merging:

In [93]:
#the id_vehicule will be the year+index :
for year in range(2005, 2019):
    locals()["veh"+"_"+str(year)[-2:]] ['id_vehicule'] = (str(year) + locals()["veh"+"_"+str(year)[-2:]].index.astype('str')).astype('int64')

* We change type of *id_vehicule* in 2019-2020 to *int*:

In [94]:
#correcting the encoding problem within id_column in 2019-2020 : 
veh_19['id_vehicule'] = veh_19['id_vehicule'].str.encode('ascii', 'ignore').str.decode('UTF-8')
veh_20['id_vehicule'] = veh_20['id_vehicule'].str.encode('ascii', 'ignore').str.decode('UTF-8')

#convert to int64
veh_19['id_vehicule'] = veh_19['id_vehicule'].astype('int64')
veh_20['id_vehicule'] = veh_20['id_vehicule'].astype('int64')

* We adapt *manv* column in 2019-2020 with the years=<2018. We replace 26 and 25 with 0 , the two values have very limited number and can be considered a part of the 0 (unkown) category in the other years:

In [95]:
veh_19.manv.replace([25,26], 0, inplace=True)
veh_20.manv.replace([25,26], 0, inplace=True)

- We combine between *obsm* & *obs*. The two columns are going to be combined in *obs* (*obsm* is dropped):
   - for the fixed obstacle obs=10
   - for the multi obstacle presence obs=11
   - we combine obsm=5 & obsm=6  in obs=5 (corresponds to diffrent animals)
   - we combine obsm=2 & obsm=4 in obs=2 (correponds to diffrent vehicles)
   - we keep obsm=9 as obs=9

In [96]:
for year in range(2005, 2021):

    #replace all the fixed obstacle with value 10
    locals()["veh"+"_"+str(year)[-2:]].loc[(locals()["veh"+"_"+str(year)[-2:]].obs!=0) & (locals()["veh"+"_"+str(year)[-2:]].obsm==0), 'obs']= 10
    
    #if obsm!=0 et ob!=0 means that there's multiobstacles, we give the value 11:
    locals()["veh"+"_"+str(year)[-2:]].loc[locals()["veh"+"_"+str(year)[-2:]].obs==11 , 'obs']= -1
    locals()["veh"+"_"+str(year)[-2:]].loc[(locals()["veh"+"_"+str(year)[-2:]].obs!=0) & (locals()["veh"+"_"+str(year)[-2:]].obsm!=0),'obs']= 11

    #we combine between obsm=5 and obsm=6, in obs=5
    locals()["veh"+"_"+str(year)[-2:]].loc[(locals()["veh"+"_"+str(year)[-2:]].obsm==6) & (locals()["veh"+"_"+str(year)[-2:]].obs==0),'obs']= 5
    #we combine between obsm=2 and obsm=4, in obs=2
    locals()["veh"+"_"+str(year)[-2:]].loc[(locals()["veh"+"_"+str(year)[-2:]].obsm==4) & (locals()["veh"+"_"+str(year)[-2:]].obs==0),'obs']= 2
    
    #obs=9 for obsm=9
    locals()["veh"+"_"+str(year)[-2:]].loc[(locals()["veh"+"_"+str(year)[-2:]].obsm==9) & (locals()["veh"+"_"+str(year)[-2:]].obs==0),'obs']= 9
    #we drop the obsm
    locals()["veh"+"_"+str(year)[-2:]].drop('obsm', axis=1, inplace= True )


* We drop the *motor* column in 2019-2020, since we won't use it :

In [97]:
veh_19.drop('motor', axis=1, inplace=True)
veh_20.drop('motor', axis=1, inplace=True)

* We drop *occutc* column since most values are not specified and is not useful in our use case. *senc* column is alos useless in our case, so we drop it too:

In [98]:
for year in range(2005, 2021):
    locals()["veh"+"_"+str(year)[-2:]].drop(['occutc','senc'],  axis=1, inplace= True)

## Merging files by year for : *Usagers* & *Vehicules*

In [99]:
#init
usagers= user_05
vehicules= veh_05

#concatenate files for usage
for year in range(2006, 2021):
    usagers= pd.concat([usagers, locals()["user"+"_"+str(year)[-2:]]  ])
    vehicules= pd.concat([vehicules, locals()["veh"+"_"+str(year)[-2:]] ] )

- Adding a column for ***usagers_id***. This will make the understanding of the relation between the diffrent tables clearer:

In [106]:

#create a column for usager_id
usagers['usager_id']= usagers.index

#place the new columns first
cols = usagers.columns.tolist()
cols = cols[-1:] + cols[:-1]
usagers= usagers[cols]

#Replace NaN with -1 : 
usagers.fillna(-1, inplace=True)
vehicules.fillna(-1, inplace=True)

#Change float64 to int64 :
usagers= usagers.astype({'trajet':'int64', 'an_nais':'int64', 'place':'int64'})
vehicules= vehicules.astype({'catv':'int64', 'obs':'int64', 'choc':'int64','manv':'int64'})

## Export generated tables to new files : *Usagers.csv* & *Vehicule.csv*

- The generated tables are saved to the ***data/*** folder :

In [ ]:
usagers.to_csv(path_or_buf= 'data/Usagers.csv',sep=';')
vehicules.to_csv(path_or_buf= 'data/Vehicules.csv',sep=';')

In [ ]:
#to test
#x=pd.read_csv('data/vehicules.csv', sep=';', index_col=0)
#y=pd.read_csv('data/usagers.csv', sep=';', index_col=0)

#x.dtypes
#y.dtypes